# Research Graph

<a href="https://colab.research.google.com/github/auto-res/researchgraph/blob/main/notebook/sample_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://github.com/auto-res/researchgraph

In [ ]:
import os
from google.colab import userdata

# Please register the secret with the name below in google colab.

# https://platform.openai.com/settings/organization/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# https://app.devin.ai/settings/api-keys
os.environ["DEVIN_API_KEY"] = userdata.get("DEVIN_API_KEY")
# https://www.firecrawl.dev/app/api-keys
os.environ["FIRE_CRAWL_API_KEY"] = userdata.get("FIRE_CRAWL_API_KEY")
# https://docs.github.com/ja/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#fine-grained-personal-access-token-%E3%81%AE%E4%BD%9C%E6%88%90
os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"] = userdata.get("GITHUB_PERSONAL_ACCESS_TOKEN")
os.environ["VERTEX_AI_API_KEY"] = userdata.get("VERTEX_AI_API_KEY")

In [ ]:
!git clone --branch feature/#332-notebook-update https://github.com/auto-res/researchgraph.git

Cloning into 'researchgraph'...
remote: Enumerating objects: 5802, done.
remote: Counting objects: 100% (643/643), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 5802 (delta 328), reused 319 (delta 271), pack-reused 5159 (from 2)
Receiving objects: 100% (5802/5802), 34.09 MiB | 32.90 MiB/s, done.
Resolving deltas: 100% (3210/3210), done.


In [ ]:
%%capture
%cd /content/researchgraph
%pip install -r requirements.txt

In [ ]:
import glob

import sys
sys.path.append('/content/researchgraph/src')

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s: %(message)s'
)

In [ ]:
github_repository = "auto-res2/test26"
branch_name = "test"
save_dir = "/workspaces/researchgraph/data"

### リポジトリの準備

In [ ]:
from researchgraph.research_preparation_subgraph.research_preparation_subgraph import ResearchPreparationSubgraph

In [ ]:
# 修正する
research_preparation_input_data = {
    "github_owner": "auto-res2",
    "repository_name": "test26",
    "branch_name": "test",
    "device_type": "gpu",
    "organization": "auto-res2",
}

result = ResearchPreparationSubgraph().run(research_preparation_input_data)
print(result)

[INFO] researchgraph.utils.execution_timers: [research_preparation._get_github_repository] Start
[INFO] researchgraph.research_preparation_subgraph.research_preparation_subgraph: ---ResearchPreparation---
[INFO] researchgraph.research_preparation_subgraph.nodes.check_github_repository: Repository not found: https://api.github.com/repos/auto-res2/test26
[INFO] researchgraph.utils.execution_timers: [research_preparation._get_github_repository] End    Execution Time:  0.1289 seconds
[INFO] researchgraph.utils.execution_timers: [research_preparation._fork_repository]  Start
[INFO] researchgraph.research_preparation_subgraph.nodes.fork_repository: Fork of the repository was successful.
[INFO] researchgraph.utils.execution_timers: [research_preparation._fork_repository]  End    Execution Time:  0.6010 seconds
[INFO] researchgraph.utils.execution_timers: [research_preparation._retrieve_branch_name] Start
[INFO] researchgraph.research_preparation_subgraph.nodes.check_branch_existence: Branch n

{'github_owner': 'auto-res2', 'repository_name': 'test26', 'branch_name': 'test', 'device_type': 'gpu', 'organization': 'auto-res2', 'repository_exists': False, 'fork_result': True, 'target_branch_sha': '', 'create_result': True, 'main_sha': '0b4ffd87d989e369a03fce523be014bc6cf75ea8'}


# 論文の取得

In [ ]:
from researchgraph.retrieve_paper_subgraph.retrieve_paper_subgraph import Retriever

In [ ]:
scrape_urls = [
    "https://icml.cc/virtual/2024/papers.html?filter=title",
    "https://iclr.cc/virtual/2024/papers.html?filter=title",
    # "https://nips.cc/virtual/2024/papers.html?filter=title",
    # "https://cvpr.thecvf.com/virtual/2024/papers.html?filter=title",
]
add_paper_num = 1

retriever = Retriever(
    github_repository=github_repository,
    branch_name=branch_name,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
    scrape_urls=scrape_urls,
    add_paper_num=add_paper_num,
)

retriever_input = {
    "queries": ["diffusion model"],
}

result = retriever.run(retriever_input)
print(result)

# 新規手法の生成

In [ ]:
from researchgraph.generator_subgraph.generator_subgraph import Generator

In [ ]:
retriever = Generator(
    github_repository=github_repository,
    branch_name=branch_name,
    llm_name="o1-2024-12-17",
)

result = retriever.run()
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [generator_subgraph._generator_node]     Start
[INFO] researchgraph.generator_subgraph.generator_subgraph: ---GeneratorSubgraph---
[INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
[INFO] researchgraph.utils.execution_timers: [generator_subgraph._generator_node]     End    Execution Time: 21.5218 seconds
[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Uploading state to: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler

result: {'github_upload_success': True}


# 実験コードの生成

In [ ]:
from researchgraph.experimental_plan_subgraph.experimental_plan_subgraph import ExperimentalPlaner

In [ ]:
experimentalplaner = ExperimentalPlaner(
    github_repository=github_repository,
    branch_name=branch_name,
)

result = experimentalplaner.run()
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [experimental_plan_subgraph._generate_advantage_criteria_node] Start
[INFO] researchgraph.utils.execution_timers: [experimental_plan_subgraph._retrieve_code_with_devin_node] Start
[INFO] researchgraph.experimental_plan_subgraph.experimental_plan_subgraph: ---ExperimentalPlanSubgraph---
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.devin.ai/v1/sessions
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.experime

result: {'github_upload_success': True}


# 実験の実行

In [ ]:
from researchgraph.executor_subgraph.executor_subgraph import Executor

In [ ]:
max_code_fix_iteratio = 3

retriever = Executor(
    github_repository=github_repository,
    branch_name=branch_name,
    save_dir=save_dir,
    max_code_fix_iteration=max_code_fix_iteratio,
)

result = retriever.run()
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [executor_subgraph._generate_code_with_devin_node] Start
[INFO] researchgraph.executor_subgraph.executor_subgraph: ---ExecutorSubgraph---
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.devin.ai/v1/sessions
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.executor_subgraph.nodes.generate_code_with_devin: Successfully created Devin session.
[INFO] researchgraph.executor_subgraph.nodes.generate_code_with_devin: 

result: {'github_upload_success': True}


# 実験結果の分析

In [ ]:
from researchgraph.analytic_subgraph.analytic_subgraph import Analyst

In [ ]:
retriever = Analyst(
    github_repository=github_repository, branch_name=branch_name, llm_name="o1-2024-12-17"
)

result = retriever.run()
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [analytic_subgraph._analytic_node]       Start
[INFO] researchgraph.analytic_subgraph.analytic_subgraph: ---AnalyticSubgraph---
[INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] researchgraph.utils.execution_timers: [analytic_subgraph._analytic_node]       End    Execution Time: 27.7859 seconds
[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Uploading state to: .research/research_history.json
[INFO] researchgraph.utils.api_request_han

result: {'github_upload_success': True}


# 論文の執筆

In [ ]:
from researchgraph.writer_subgraph.writer_subgraph import PaperWriter

In [ ]:
refine_round = 1

paper_writer = PaperWriter(
    github_repository=github_repository,
    branch_name=branch_name,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
    refine_round=refine_round,
)

result = paper_writer.run({})
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [writer_subgraph._generate_note_node]    Start
[INFO] researchgraph.writer_subgraph.writer_subgraph: ---WriterSubgraph---
[INFO] researchgraph.utils.execution_timers: [writer_subgraph._generate_note_node]    End    Execution Time:  0.0072 seconds
[INFO] researchgraph.utils.execution_timers: [writer_subgraph._writeup_node]          Start
[INFO] researchgraph.writer_subgraph.nodes.paper_writing: Generating full paper in one LLM call...
[INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/co

result: {'github_upload_success': True}


# 研究結果をREADME.mdにまとめる

In [ ]:
from researchgraph.readme_subgraph.readme_subgraph import ReadmeUploader

ModuleNotFoundError: No module named 'researchgraph.upload_subgraph'

In [ ]:
readme_uploader = ReadmeUploader(
    github_repository=github_repository,
    branch_name=branch_name,
)

result = readme_uploader.run()
print(f"result: {result}")

# 論文をHTMLで出力

In [ ]:
from researchgraph.html_subgraph.html_subgraph import HtmlConverter

In [ ]:
figures_dir = f"{save_dir}/images"
pdf_files = glob.glob(os.path.join(figures_dir, "*.pdf"))

extra_files = [
    {
        "upload_branch": "gh-pages",
        "upload_dir": "branches/{{ branch_name }}/",
        "local_file_paths": [f"{save_dir}/index.html"],
    },
    {
        "upload_branch": "gh-pages",
        "upload_dir": "branches/{{ branch_name }}/images/",
        "local_file_paths": pdf_files,
    },
]

html_converter = HtmlConverter(
    github_repository=github_repository,
    branch_name=branch_name,
    extra_files=extra_files,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
)

result = html_converter.run()
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [html_subgraph._convert_to_html_node]    Start
[INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] researchgraph.utils.execution_timers: [html_subgraph._convert_to_html_node]    End    Execution Time: 21.0368 seconds
[INFO] researchgraph.utils.execution_timers: [html_subgraph._render_html_node]        Start
[INFO] researchgraph.html_subgraph.nodes.render_html: Saved HTML to: /content/researchgraph/content/researchgraph/data/index.html
[INFO] researchg

result: {'github_upload_success': True}


# 論文をPDFで出力

In [ ]:
!apt-get update && apt-get install -y \
    texlive-base \
    texlive-latex-recommended \
    texlive-fonts-recommended \
    texlive-latex-extra \
    texlive-science \
    chktex \
    locales && \
    rm -rf /var/lib/apt/lists/*

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,697 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,844 kB]
Hit:13 http://archive.ubuntu.com/ubuntu jamm

In [ ]:
from researchgraph.latex_subgraph.latex_subgraph import LatexConverter

In [ ]:
extra_files = [
    {
        "upload_branch": "{{ branch_name }}",
        "upload_dir": ".research/",
        "local_file_paths": [f"{save_dir}/paper.pdf"],
    }
]

latex_converter = LatexConverter(
    github_repository=github_repository,
    branch_name=branch_name,
    extra_files=extra_files,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
)

result = latex_converter.run({})
print(f"result: {result}")

[INFO] researchgraph.github_utils.github_file_io: [GitHub I/O] Downloading input from: .research/research_history.json
[INFO] researchgraph.utils.api_request_handler: Requests to endpoints:https://api.github.com/repos/auto-res2/test26/contents/.research/research_history.json
[INFO] researchgraph.utils.api_request_handler: API request successful on attempt 1.
[INFO] researchgraph.github_utils.graph_wrapper: No conflict. Using existing branch: test
[INFO] researchgraph.utils.execution_timers: [latex_subgraph._latex_node]             Start
[INFO] httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] researchgraph.utils.execution_timers: [latex_subgraph._latex_node]             End    Execution Time: 25.9399 seconds
[INFO] researchgraph.utils.execution_timers: [latex_subgraph._latex_node]             Start
[INFO] researchgraph.latex_subgraph.nodes.compile_to_pdf: 置換完了: TITLE HERE
[INFO] researchgraph.latex_subgraph.nodes.compile_to_pdf: 置換完了: ABSTRAC

result: {'github_upload_success': True}
